In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data_UMAPDA.pkl


In [2]:
import joblib
path_ = '/kaggle/input/data_UMAPDA.pkl'

dataset = joblib.load(path_) 

In [3]:
dataset.keys()
#ind_total_aug thresholds vs Datasets

dict_keys(['Xc', 'Yc', 'names_', 'label_dat', 'max_', 'ths', 'ind_total', 'ind_source', 'ind_target', 'ind_total_aug'])

In [4]:
Xc = dataset['Xc']
Yc = dataset['Yc']
label_dat = dataset['label_dat'].ravel()
names_ = dataset['names_']
ths = dataset['ths']
ind_total_aug = dataset['ind_total_aug']

print(Xc.shape,Yc.shape)

(162036, 20) (162036, 7)


In [5]:
names_

['Argone', 'Beijing', 'Chengdu']

# Original data

In [6]:
from sklearn.model_selection import train_test_split
train_size = 0.8
val_size = 0.2
random_state = 123

#### cambiar dataset y ths
datai = 1 # 1: Argone, 2: Beijing, 3: Chengdu
thj = 0 # 0: 0.001, 1: 0.002, 2: 0.005, 3: 0.007, 4: 0.01, 5:  0.02

X_ = Xc[label_dat==datai]
Y_ = Yc[label_dat==datai]
nsamples = X_.shape[0]
Xtrain_ = X_[:int(nsamples*train_size)]
Xtest = X_[int(nsamples*train_size):]
Ytrain_ = Y_[:int(nsamples*train_size)]
Ytest = Y_[int(nsamples*train_size):]

Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_,Ytrain_, test_size=val_size, 
                                              random_state=random_state, shuffle=True)



# Data augmentation

In [7]:
vec_ = ind_total_aug[thj][datai]
Xtrain_a = np.r_[Xtrain,Xc[vec_]]
Ytrain_a = np.r_[Ytrain,Yc[vec_]]

print(f"Augmented samples {len(vec_)}")

print(f"All: {X_.shape},{Y_.shape}")
print(f"Train_o: {Xtrain.shape},{Ytrain.shape}") # datos sin data augmentation
print(f"Train_a: {Xtrain_a.shape},{Ytrain_a.shape}--Th={ths[thj]}") #datos para entrenar con data augmentation segun umbral thj
print(f"Val: {Xval.shape},{Yval.shape}")
print(f"Test: {Xtest.shape},{Ytest.shape}")

Augmented samples 2400
All: (54315, 20),(54315, 7)
Train_o: (34761, 20),(34761, 7)
Train_a: (37161, 20),(37161, 7)--Th=0.001
Val: (8691, 20),(8691, 7)
Test: (10863, 20),(10863, 7)


# Train and test network

In [8]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from tabulate import tabulate
from sklearn.model_selection import TimeSeriesSplit

# Configurar TensorFlow para entrenamiento en múltiples GPUs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Función de pérdida personalizada: Kernel MSE
def kernel_mse_loss(y_true, y_pred):
    sigma = tf.sqrt(2.0) / 2.0
    diff = tf.square(y_true - y_pred)
    return tf.reduce_mean(1.0 - tf.exp(-diff / (2.0 * sigma**2)))

# Función MAPE personalizada con umbral
#def calculate_mape(y_true, y_pred, threshold=1e-2):
#    y_true_safe = np.where(np.abs(y_true) < threshold, threshold, y_true)
#    return mean_absolute_percentage_error(y_true_safe, y_pred) * 100

def calculate_mape(y_true, y_pred, threshold=1e-2):
    # Creamos una máscara para los valores de y_true que sean suficientemente grandes
    mask = np.abs(y_true) >= threshold
    if not np.any(mask):
        raise ValueError("Ningún valor en y_true cumple la condición de ser mayor o igual al threshold")
    # Calculamos el MAPE usando únicamente los valores que pasan la máscara
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / np.abs(y_true[mask]))) * 100
    return mape

In [9]:
class ForecastingModels:
    def __init__(self, X_train, y_train, X_valid, y_valid, X_test, y_test, neurons, num_layers, batch_size, epochs, loss_function): 

        if len(X_train.shape) == 2:
            X_train = np.expand_dims(X_train, axis=-1)
        if len(X_valid.shape) == 2:
            X_valid = np.expand_dims(X_valid, axis=-1)
        if len(X_test.shape) == 2:
            X_test = np.expand_dims(X_test, axis=-1)
        
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.X_test = X_test        
        self.y_test = y_test
        self.neurons = neurons
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.epochs = epochs  
        self.predictionHorizon = self.y_train.shape[1] if len(self.y_train.shape) > 1 else 1

        # Selección de la función de pérdida
        loss_functions = {
            "mse": "mse",
            "mae": "mae",
            "KernelMSE": kernel_mse_loss
        }

        if loss_function not in loss_functions:
            raise ValueError("Invalid loss function. Choose 'mse', 'mae', or 'KernelMSE'")

        self.loss_function = loss_functions[loss_function]

    def Performance_Metrics(self, forecasting_test):
        metrics = {"MSE": [], "RMSE": [], "MAE": [], "MAPE": [], "R2": []}
        global_metrics = {"MSE Global": [], "RMSE Global": [], "MAE Global": [], "MAPE Global": [], "R2 Global": []}

        for k in range(self.predictionHorizon):
            y_true, y_pred = self.y_test[:, k]*40.23, forecasting_test[:, k]*40.23

            metrics["MSE"].append(round(mean_squared_error(y_true, y_pred), 3))
            metrics["RMSE"].append(round(np.sqrt(mean_squared_error(y_true, y_pred)), 3))
            metrics["MAE"].append(round(mean_absolute_error(y_true, y_pred), 3))
            metrics["MAPE"].append(round(calculate_mape(y_true, y_pred), 3))
            metrics["R2"].append(round(r2_score(y_true, y_pred), 3))

            # Métricas acumuladas sin flatten()
            #y_true_cumulative = self.y_test[:, :k].ravel()
            #y_pred_cumulative = forecasting_test[:, :k].ravel()*40.23
            y_true_cumulative = self.y_test[:, :k+1].ravel()
            y_pred_cumulative = forecasting_test[:, :k+1].ravel()

            
            if y_true_cumulative.size == 0 or y_pred_cumulative.size == 0:
                continue
    
            global_metrics["MSE Global"].append(mean_squared_error(y_true_cumulative, y_pred_cumulative))
            global_metrics["RMSE Global"].append(np.sqrt(global_metrics["MSE Global"][-1]))
            global_metrics["MAE Global"].append(mean_absolute_error(y_true_cumulative, y_pred_cumulative))
            global_metrics["MAPE Global"].append(calculate_mape(y_true_cumulative, y_pred_cumulative))
            global_metrics["R2 Global"].append(r2_score(y_true_cumulative, y_pred_cumulative))
            
        return metrics, global_metrics

    def build_and_train_model(self, model_type, X_train, y_train, X_valid, y_valid):
        strategy = tf.distribute.MirroredStrategy()
        with strategy.scope():
            input_layer = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]), name='Input')
            layer = input_layer

            for i in range(self.num_layers):
                return_sequences = i < self.num_layers - 1
                RNNLayer = {"RNN": keras.layers.SimpleRNN,
                            "GRU": keras.layers.GRU,
                            "LSTM": keras.layers.LSTM}.get(model_type)

                if not RNNLayer:
                    raise ValueError("Invalid model type")

                layer = RNNLayer(self.neurons, activation='relu', return_sequences=return_sequences, name=f'h{i+1}')(layer)

            output_layer = keras.layers.Dense(self.predictionHorizon, name='Output')(layer)
            model = keras.Model(inputs=input_layer, outputs=output_layer)
            model.compile(loss=self.loss_function, optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['mae', 'mse'])
            history = model.fit(X_train, y_train, epochs=self.epochs, batch_size=self.batch_size, validation_data=(X_valid, y_valid), verbose=0)
        
        return model, history

    def train_model(self, model_type):
        model, history = self.build_and_train_model(model_type, self.X_train, self.y_train, self.X_valid, self.y_valid)
        y_pred = model.predict(self.X_test)
        metrics, global_metrics = self.Performance_Metrics(y_pred)
        return metrics, global_metrics, history, y_pred

    def RNNSimple_Model(self):
        return self.train_model('RNN')

    def GRU_Model(self):
        return self.train_model('GRU')

    def LSTM_Model(self):
        return self.train_model('LSTM')

In [10]:
Xtrain_a.shape, Ytrain_a.shape, Xval.shape, Yval.shape, Xtest.shape, Ytest.shape

((37161, 20), (37161, 7), (8691, 20), (8691, 7), (10863, 20), (10863, 7))

In [11]:
### Aca entrenar y evaluar redes, dejando fijos los xtrain_a, xval, xtest y las respectivas salidas tal y como las fijamos
# en la celda anterior. 

# para cambiar de dataset y umbral cambiar datai y thj
# datai = 1 # 1: Argone, 2: Beijing, 3: Chengdu
# thj = 0 # 0: 0.001, 1: 0.002, 2: 0.005, 3: 0.007, 4: 0.01, 5:  0.02


In [12]:
import time
import matplotlib.pyplot as plt 

In [13]:
names_

['Argone', 'Beijing', 'Chengdu']

In [14]:
#loop
thsA = np.append(0,ths) #include original data
thsA
train_size = 0.8
val_size = 0.2
random_state = 123

results = {}
for datai in range(len(names_)):
    for thj in range(len(thsA)): 
        print(f"\n\t {names_[datai]}--th={thsA[thj].round(3)}")
        
        X_ = Xc[label_dat==datai+1]
        Y_ = Yc[label_dat==datai+1]
        nsamples = X_.shape[0]
        Xtrain_ = X_[:int(nsamples*train_size)]
        Xtest = X_[int(nsamples*train_size):]
        Ytrain_ = Y_[:int(nsamples*train_size)]
        Ytest = Y_[int(nsamples*train_size):]

        Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_,Ytrain_, test_size=val_size, 
                                              random_state=random_state, shuffle=True)
        if thsA[thj] > 0:
            vec_ = ind_total_aug[thj-1][datai]
            Xtrain_a = np.r_[Xtrain,Xc[vec_]]
            Ytrain_a = np.r_[Ytrain,Yc[vec_]]
        else:
            vec_ = np.empty(0)
            Xtrain_a = Xtrain
            Ytrain_a = Ytrain

        pda = (((Xtrain_a.shape[0]-Xtrain.shape[0])/Xtrain.shape[0])*100)
        print(f"Augmented samples {len(vec_)}-- per DA={round(pda,1)}%")
        
        print(f"All: {X_.shape},{Y_.shape}")
        print(f"Train_o: {Xtrain.shape},{Ytrain.shape}")     # datos sin data augmentation
        print(f"Train_a: {Xtrain_a.shape},{Ytrain_a.shape}") # datos para entrenar con data augmentation segun umbral thj
        print(f"Val: {Xval.shape},{Yval.shape}")
        print(f"Test: {Xtest.shape},{Ytest.shape}")
        
        ####train your network here
        model = ForecastingModels(Xtrain_a, Ytrain_a, Xval, Yval, Xtest, Ytest, 
                          neurons=9, num_layers=1, batch_size=256, epochs=100, loss_function='KernelMSE') 

        # Medir tiempo de RNNSimple_Model
        # Entrenar y medir tiempo de cada modelo
        
        metrics = {}
        for model_name, model_func in zip(["RNNSimple_Model", "GRU_Model", "LSTM_Model"],[model.RNNSimple_Model, model.GRU_Model, model.LSTM_Model]):
            start = time.perf_counter()
            metrics_model, global_metrics_model, history, y_pred = model_func()
            end = time.perf_counter()
            
            metrics[model_name] = {
                "Point": metrics_model,
                "Global": global_metrics_model,
                "Ypred": y_pred, 
                "History": history
            }
            
            print(global_metrics_model['R2 Global'], global_metrics_model['MAPE Global'], global_metrics_model['MAE Global'])
            print(f"Time {model_name}: {end - start} [s]")

        # Guardar resultados en diccionario
        results[(names_[datai], thsA[thj].round(3))] = metrics
        



	 Argone--th=0.0
Augmented samples 0-- per DA=0.0%
All: (54315, 20),(54315, 7)
Train_o: (34761, 20),(34761, 7)
Train_a: (34761, 20),(34761, 7)
Val: (8691, 20),(8691, 7)
Test: (10863, 20),(10863, 7)
340/340 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step
[0.8873590019287748, 0.8281584554923281, 0.7697555943119194, 0.7092175026593621, 0.6509177611950987, 0.5938820637619648, 0.5365492555453313] [16.439430378478924, 20.11454561306297, 23.132127555713534, 25.7747425746221, 28.002009944517326, 29.936215297934766, 31.608100202029952] [1.0155812135866902, 1.2387168367069465, 1.4259453912404483, 1.595571533401489, 1.7451916952234139, 1.8810820443992577, 2.007528858024031]
Time RNNSimple_Model: 244.39619396100002 [s]
340/340 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
[0.890838680301848, 0.8293218882814314, 0.7688183897837595, 0.7100947435633223, 0.65162075139503, 0.596192062448793, 0.5398832974565446] [15.820364781456666, 19.63918929452579, 22.824187585772666, 25.53795224072639, 27.80521748304155, 29.78910011479421, 

In [15]:
import pickle
with open("/kaggle/working/Results_2.pkl", "wb") as f:
    pickle.dump(results, f)